Experiment 2: Fairness Mitigation with SMOTE on Bank Marketing Data

Dataset:
Bank Marketing (UCI) — contains demographic, contact, and economic details about clients contacted for a term deposit marketing campaign.

Sensitive attribute: Age group (younger vs older, split at median age).

Target variable: deposit (1 = subscribed, 0 = not subscribed).

Privileged group: Older clients (age ≥ median).

Unprivileged group: Younger clients (age < median).

Task: Predict deposit subscription while reducing disparities between age groups.

Installation and Imports

In [1]:
%pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 3.7 MB/s eta 0:00:00


In [2]:
pip install aif360

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 3.3 MB/s eta 0:00:00


In [3]:
pip install aif360[inFairness]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import zipfile
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, false_positive_rate, false_negative_rate

Load and Preprocess Data

In [6]:
zip_file_path = "/content/bank+marketing (2).zip"

with zipfile.ZipFile(zip_file_path, 'r') as z:
    with z.open("bank-additional.zip") as inner_zip_file:
        with zipfile.ZipFile(inner_zip_file, 'r') as inner_z:
            with inner_z.open("bank-additional/bank-additional-full.csv") as f:
                df = pd.read_csv(f, sep=';')

df = df.dropna()
df['deposit'] = (df['y'] == 'yes').astype(int)

Creation of Binary Age Group

In [7]:
median_age = df['age'].median()
df['age_group'] = (df['age'] >= median_age).astype(int)

One-Hot Encode Categorical Features

In [8]:
categorical_cols = [
    'job', 'marital', 'education', 'default', 'housing',
    'loan', 'contact', 'month', 'poutcome', 'day_of_week'
]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

Features, Labels, Sensitive Attribute

In [9]:
features = [col for col in df.columns if col not in ['y', 'deposit', 'age_group', 'age']]
X = df[features]
y = df['deposit']
sensitive_features_col = ['age_group']
sensitive_features = df[sensitive_features_col]

Train-Test Split

In [10]:
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
    X, y, sensitive_features, test_size=0.3, random_state=42, stratify=y
)

Apply SMOTE on Training Set

In [11]:
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

Scale Data

In [12]:
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_bal)
X_test_scaled = scaler.transform(X_test)

Train Logistic Regression

In [13]:
model = LogisticRegression(solver='saga', max_iter=5000, random_state=42)
model.fit(X_train_scaled, y_train_bal)

LogisticRegression(max_iter=5000, random_state=42, solver='saga')

Fairness Evaluation

In [14]:
y_pred = model.predict(X_test_scaled)

mf = MetricFrame(
    metrics={
        'TPR': true_positive_rate,
        'FPR': false_positive_rate,
        'FNR': false_negative_rate,
        'Selection Rate': selection_rate
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=s_test['age_group']
)

print("Fairness Metrics by Age Group After SMOTE:\n")
print(mf.by_group)
print("\nOverall Metrics:\n")
print(mf.overall)

Fairness Metrics by Age Group After SMOTE:

                TPR       FPR       FNR  Selection Rate
age_group                                              
0          0.582173  0.062083  0.417827        0.124644
1          0.548961  0.047077  0.451039        0.100031

Overall Metrics:

TPR               0.566092
FPR               0.054264
FNR               0.433908
Selection Rate    0.111920
dtype: float64
